In [7]:
from tqdm import tqdm
import os

audio_dir = "/Users/haohan/Documents/ai/tednet/data"
save_dir = "/Users/haohan/Documents/ai/tednet/save"

with open("/Users/haohan/Documents/ai/tednet/data/metadata.csv", "w") as f:
    f.write("file_name,transcription\n")
    for home, dirs, files in tqdm(os.walk(audio_dir)):
        for filename in files:
            if filename.endswith(".wav"):
                f.write(f"wav/{filename},")
                f.write("dskjfk dsfds\n")


2it [00:00, 2628.01it/s]


In [19]:
from datasets import load_dataset
common_voice = load_dataset("audiofolder", data_dir="/Users/haohan/Documents/ai/tednet/save/clean")

Resolving data files:   0%|          | 0/101 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(common_voice)

DatasetDict({
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 16354
    })
})


In [3]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice = load_dataset("mozilla-foundation/common_voice_11_0", "zh-TW", split="train+validation+test", cache_dir="./zh-TW/")
# common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test")

print(common_voice)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 15986
})


In [6]:
common_voice.save_to_disk("../zh-TW")

Saving the dataset (1/1 shards): 100%|██████████| 15986/15986 [00:01<00:00, 9743.41 examples/s] 


In [79]:
from datasets import Audio

common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
print(common_voice)

DatasetDict({
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 16354
    })
})


In [ ]:
common_voice["test"][4]

{'audio': {'path': '/Users/icerm/.cache/huggingface/datasets/downloads/extracted/ba51f659b4fc7c756f2b2378baee4e571fcbade733a825e299fbfdb686213f48/en_test_0/common_voice_en_55485.mp3',
  'array': array([ 5.82076609e-11, -8.00355338e-11,  5.82076609e-11, ...,
         -4.18658601e-07,  1.51124550e-06,  5.98614861e-06]),
  'sampling_rate': 16000},
 'sentence': '"Like everybody learns," he said.'}

In [53]:
from datasets import load_dataset
common_voice = load_dataset("audiofolder", data_dir="/Users/icerm/Documents/ai/tednet/save/10db")
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 955
    })
})


In [54]:
from datasets import load_dataset, Audio
import torch
import pandas as pd
import re
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import evaluate

device = 'cuda' if torch.cuda.is_available() else 'mps'
print('Inference device:', device)

dummy = common_voice

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language='en', task='transcribe')
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to(device)


results = []
for i in range(955):
    print(f"label {i} : {dummy['train'][i]['sentence']}")
    inputs = processor(dummy['train'][i]["audio"]["array"], return_tensors="pt", sampling_rate=16000)
    input_features = inputs.input_features

    generated_ids = model.generate(inputs=input_features.to(device), task='transcribe', language='en')

    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    string = ''.join(transcription[1:])
    print('trans', i, ":", transcription[1:])
    results.append(string)

chars_to_remove_regex = r'[\,\'\.\-\,\۔\!\,\؟\:\-\“\%\‘\’\”\،\"]'
predictions = [re.sub(chars_to_remove_regex, '', str(x).lower()) for x in results]

references = [dummy['train'][i]['sentence'].lower() for i in range(955)]

wer_metric = evaluate.load("wer")

wer_score = wer_metric.compute(predictions=predictions, references=references)
print("wer score", wer_score)

Inference device: mps
label 0 : Joe Keaton disapproved of films, and Buster also had reservations about the medium.
trans 0 : Joel Keaton disapproved of films and Buster also had reservations about the media.
label 1 : She'll be all right.
trans 1 : Should be alright
label 2 : No one near the remote machine you could ring?
trans 2 : No one near the remote machine you could ring.
label 3 : George Hare and Rev.
trans 3 : Yours, here, are well.
label 4 : He took the fourth part and handed it to the monk.
trans 4 : He took the first part and needed to do the mark.
label 5 : He is a right-handed batsman and a left-arm slow bowler.
trans 5 : He is a right handed batsman and the left arm slow bowler.
label 6 : Cador is also said to be of Roman stock.
trans 6 : Cattle is also said to be Roman stock.
label 7 : Where will you go?
trans 7 : Where will you go?
label 8 : Only John Wayne himself surpassed McGrath in scenes in this movie.
trans 8 : and you turn right and set the fast mag blade in thi

In [11]:
wer_metric = evaluate
wer_metric = wer_metric.load("wer")
l= 1
wer_score = wer_metric.compute(predictions=predictions[:l], references=references[:l])
print("wer score", wer_score)

wer score 0.15384615384615385


In [24]:
predictions

['joe keaton disapproved of films and buster also had reservations about the media',
 'should be alright',
 'no one near the remote machine you could ring',
 'yours here and well',
 'he took the first heart and handed it to the monk',
 'he is a righthanded batsman and a leftarm slow bowler',
 'kado is also said to be of roman stock',
 'where will you go?',
 'only john wayne can set such a fast mac blurred in scenes in this movie',
 'it was opened in july of that year',
 'rushling is a talented fan manager she works in vienna',
 'they were eliminated in the fourth episode',
 'munisin means near the island',
 'sauce for the goose is sauce for the gander',
 'the owner of the dwelling was not immune',
 'all of this activity in milwaukee was in no way unique',
 'william barnes the poet also lived in starnister utah',
 'bye im so happy going through the steps in unison',
 'im so glad to see you',
 'corn cockle is an attractive plant and its seeds are still commercially available to gardeners

In [22]:
references

['joe keaton disapproved of films, and buster also had reservations about the medium.',
 "she'll be all right.",
 'no one near the remote machine you could ring?',
 'george hare and rev.',
 'he took the fourth part and handed it to the monk.',
 'he is a right-handed batsman and a left-arm slow bowler.',
 'cador is also said to be of roman stock.',
 'where will you go?',
 'only john wayne himself surpassed mcgrath in scenes in this movie.',
 'it was opened in july of that year.',
 'vera schlink is a talented fund manager. she works in vienna.',
 'they were eliminated in the fourth episode.',
 "'munising' means near the island.",
 'sauce for the goose is sauce for the gander.',
 'the owner of the dwelling was not amused.',
 'all of this activity in milwaukee was in no way unique.',
 'william barnes the poet also lived in sturminster newton.',
 'thai dancers happily going through their steps in unison.',
 "i'm so glad to see you.",
 'corncockle is an attractive plant, and its seeds are st

In [84]:
import os

import torchaudio
from torch_audiomentations import AddBackgroundNoise, Compose

transform0 = Compose(
    transforms = [
        AddBackgroundNoise(
            background_paths="/Users/icerm/Documents/ai/tednet/ESC-50/audio",
            min_snr_in_db=30.0,
            max_snr_in_db=30.0,
            p=1,
            output_type="tensor"
        ),
    ],
    output_type="tensor"
)
transform1 = Compose(
    transforms = [
        AddBackgroundNoise(
            background_paths="/Users/icerm/Documents/ai/tednet/ESC-50/audio",
            min_snr_in_db=20.0,
            max_snr_in_db=20.0,
            p=1,
            output_type="tensor"
        ),
    ],
    output_type="tensor"
)
transform2 = Compose(
    transforms = [
        AddBackgroundNoise(
            background_paths="/Users/icerm/Documents/ai/tednet/ESC-50/audio",
            min_snr_in_db=10.0,
            max_snr_in_db=10.0,
            p=1,
            output_type="tensor"
        ),
    ],
    output_type="tensor"
)
transform3 = Compose(
    transforms = [
        AddBackgroundNoise(
            background_paths="/Users/icerm/Documents/ai/tednet/ESC-50/audio",
            min_snr_in_db=0.0,
            max_snr_in_db=0.0,
            p=1,
            output_type="tensor"
        ),
    ],
    output_type="tensor"
)
transform4 = Compose(
    transforms = [
        AddBackgroundNoise(
            background_paths="/Users/icerm/Documents/ai/tednet/ESC-50/audio",
            min_snr_in_db=-10.0,
            max_snr_in_db=-10.0,
            p=1,
            output_type="tensor"
        ),
    ],
    output_type="tensor"
)
save_dir = "/Users/icerm/Documents/ai/tednet/save"
if not os.path.exists("/Users/icerm/Documents/ai/tednet/save/clean/wav"):
    os.makedirs("/Users/icerm/Documents/ai/tednet/save/clean/wav")
if not os.path.exists("/Users/icerm/Documents/ai/tednet/save/-10db/wav"):
    os.makedirs("/Users/icerm/Documents/ai/tednet/save/-10db/wav")
if not os.path.exists("/Users/icerm/Documents/ai/tednet/save/0db/wav"):
    os.makedirs("/Users/icerm/Documents/ai/tednet/save/0db/wav")
if not os.path.exists("/Users/icerm/Documents/ai/tednet/save/10db/wav"):
    os.makedirs("/Users/icerm/Documents/ai/tednet/save/10db/wav")
if not os.path.exists("/Users/icerm/Documents/ai/tednet/save/20db/wav"):
    os.makedirs("/Users/icerm/Documents/ai/tednet/save/20db/wav")
if not os.path.exists("/Users/icerm/Documents/ai/tednet/save/30db/wav"):
    os.makedirs("/Users/icerm/Documents/ai/tednet/save/30db/wav")
with open("/Users/icerm/Documents/ai/tednet/save/metadata.csv", "w") as f:
    f.write("file_name,sentence\n")
    for i in range(1000):
        if '"' in common_voice['test'][i]['sentence']:
            continue
        else:
            f.write(f"wav/tw{i}.wav,")
            f.write(f"\"{common_voice['test'][i]['sentence']}\"\n")
        wave, sr = torchaudio.load(common_voice['test'][i]['audio']['path'], normalize=True)
        if sr != 16000:
            wave = torchaudio.functional.resample(wave, orig_freq=sr, new_freq=16000)
        sr = 16000
        wave0 = transform0(wave.unsqueeze(0), sample_rate=sr).squeeze(0)
        wave1 = transform1(wave.unsqueeze(0), sample_rate=sr).squeeze(0)
        wave2 = transform2(wave.unsqueeze(0), sample_rate=sr).squeeze(0)
        wave3 = transform3(wave.unsqueeze(0), sample_rate=sr).squeeze(0)
        wave4 = transform4(wave.unsqueeze(0), sample_rate=sr).squeeze(0)
        torchaudio.save(os.path.join(save_dir, "clean", "wav", f"tw{i}.wav"),  wave, sr, bits_per_sample=16)
        torchaudio.save(os.path.join(save_dir,  "30db", "wav", f"tw{i}.wav"), wave0, sr, bits_per_sample=16)
        torchaudio.save(os.path.join(save_dir,  "20db", "wav", f"tw{i}.wav"), wave1, sr, bits_per_sample=16)
        torchaudio.save(os.path.join(save_dir,  "10db", "wav", f"tw{i}.wav"), wave2, sr, bits_per_sample=16)
        torchaudio.save(os.path.join(save_dir,   "0db", "wav", f"tw{i}.wav"), wave3, sr, bits_per_sample=16)
        torchaudio.save(os.path.join(save_dir, "-10db", "wav", f"tw{i}.wav"), wave4, sr, bits_per_sample=16)
        



In [76]:
dummy['train'][1]['sentence']

"'She'll be all right.'"

In [68]:
from evaluate import load

wer_metric = load("wer")

wer = wer_metric.compute(references=['No one near the remote machine you could ring?'], predictions=['No one near the remote machine you could ring?'])

print(wer)

0.0
